## Packages and Libraries Installation
If this is your first time using Jupyter Notebook and you don't have the necessary packages installed, please install these packages and libraries. You only need to install these packages once. Uncomment the cell below and run it to install the packages.


In [ ]:
# !pip install --quiet pandas numpy matplotlib 
# !pip install --quiet pymannkendall
# !pip install --quiet xclim

## Packages and Library importing
`SOBRkendallTaubPtt` and `SOBRfig2025` are pyhton packages that are written to perform Mann-Kendall/Pettitt analysis and plot the necessary figures, respectively. These functions should be in your working directory to make it easy to work. Once you keep these in the working folder where your Jupyter notebooks are saved, you can import all these functions. You have to run below cell every time you run the code.
The following are the import statements for these functions and other necessary libraries:

In [ ]:
import SOBRkendallTaubPtt as mk
import SOBRfig2025 as sb
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xclim
import os
import matplotlib.colors
import scipy.stats as st
import numpy as np
from scipy.stats import norm
import warnings; warnings.filterwarnings("ignore")

## Function Definitions for DataFrame Saving
The following functions are defined to create a DataFrame to store the results obtained from the Pettitt test and Mann-Kendall test.

In [ ]:
def determine_trend_direction(Taub):
    return "Increasing" if Taub > 0 else "Decreasing" if Taub < 0 else "No trend"

def determine_trend_significance(p):
    if p < 0.05:
        return "Highly Likely"
    elif 0.05 <= p < 0.10:
        return "Very Likely"
    elif 0.10 <= p < 0.34:
        return "Likely"
    else:
        return "Unlikely"
    
def create_mk_result_entry(station_number, trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept):
    trend_direction = determine_trend_direction(Taub)
    trend_significance = determine_trend_significance(p)

    return pd.DataFrame({
        'Station': [station_number],
        'Trend': [trend],
        'Trend Direction': [trend_direction],
        'Trend Significance': [trend_significance],
        'Hypothesis': ['trend.' if h else 'no trend.'],
        'P-MK': [round(p, 7)],
        'Z-score': [round(z, 7)],
        'Taub': [round(Taub, 7)],
        'S': [round(s, 7)],
        'Variance of S': [round(var_s, 7)],
        'Slope': [round(slope, 7)],
        'Intercept': [round(intercept, 7)],
        'Lower Bound': [lb],
        'Upper Bound': [ub]
    })

def create_mk_result_entrytime(station_number, trend, slope, h, p, z, Taub, s, var_s, lb, ub,max_day_value,corresponding_day,intercept):
    trend_direction = determine_trend_direction(slope)
    trend_significance = determine_trend_significance(p)

    return pd.DataFrame({
        'Station': [station_number],
        'Trend': [trend],
        'Trend Direction': [trend_direction],
        'Trend Significance': [trend_significance],
        'Hypothesis': ['trend.' if h else 'no trend.'],
        'P-MK': [round(p, 7)],
        'Z-score': [round(z, 7)],
        'Taub': [round(Taub, 7)],
        'S': [round(s, 7)],
        'Variance of S': [round(var_s, 7)],
        'Slope': [round(slope, 7)],
        'Intercept': [round(intercept, 7)],
        'Lower Bound': [lb],
        'Upper Bound': [ub],
        '3-day_max': [max_day_value],
        'Day_of_year': [corresponding_day]
    })

def create_mk_result_entrytime7(station_number, trend, slope, h, p, z, Taub, s, var_s, lb, ub,min_day_value,corresponding_day,intercept):
    trend_direction = determine_trend_direction(Taub)
    trend_significance = determine_trend_significance(p)

    return pd.DataFrame({
        'Station': [station_number],
        'Trend': [trend],
        'Trend Direction': [trend_direction],
        'Trend Significance': [trend_significance],
        'Hypothesis': ['trend.' if h else 'no trend.'],
        'P-MK': [round(p, 7)],
        'Z-score': [round(z, 7)],
        'Taub': [round(Taub, 7)],
        'S': [round(s, 7)],
        'Variance of S': [round(var_s, 7)],
        'Slope': [round(slope, 7)],
        'Intercept': [round(intercept, 7)],
        'Lower Bound': [lb],
        'Upper Bound': [ub],
        '7-day_min': [min_day_value],
        'Day_of_year': [corresponding_day]
    })

def create_pettitt_result_entry(station, K, p_value, locP, q):
    trend_significancePTT = determine_trend_significance(p_value)
    return pd.DataFrame({
        'Station': [station],
        'Pettitt_K': [K],
        'P-Ptt': [p_value],
        'Trend SignificancePtt': [trend_significancePTT],
        'Change value': [q.iloc[int(locP)]],
        'Change Point': [q.index[int(locP)]]
    })

def create_pettitt_result_entrytime(station, K, p_value, locP, q):
    trend_significancePTT = determine_trend_significance(p_value)
    return pd.DataFrame({
        'Station': [station],
        'Pettitt_K': [K],
        'P-Ptt': [p_value],
        'Trend SignificancePtt': [trend_significancePTT],
        'Change value': [q.iloc[int(locP)]],
        'Change Point': [q.index[int(locP)]]
    })

## ⚠️ Warning: File Path Slashes ⚠️

When specifying file paths in your code, it's important to use the correct slash and be aware of how Python interprets them:

- **Windows**: Although Windows traditionally uses backslashes (`\\`) in file paths, Python interprets a single backslash (`\`) as an escape character. This can lead to errors. To avoid this, you can use either double backslashes (`\\`) or single forward slashes (`/`). For example:

    ```python
    # Using double backslashes
    file_path = "C:\\Users\\username\\Documents\\file.txt"
    
    # Using single forward slashes
    file_path = "C:/Users/username/Documents/file.txt"
    ```

    Alternatively, you can use raw strings, which treat the `\` as a normal character rather than an escape character:

    ```python
    # Using raw strings
    file_path = r"C:\Users\username\Documents\file.txt"
    ```

- **macOS and Linux**: These systems use forward slashes (`/`) for file paths. For example:

    ```python
    file_path = "/Users/username/Documents/file.txt"
    ```

If you encounter errors when running your code, check your file paths first. Make sure you're using the correct slash, and that the file path points to the correct location.

The following files are needed to run the cells below and are provided with the code:

1. **SOBR_on84.shp**: This is the shapefile of the Ontario province that will be used to plot the spatial figure to include stations.

2. **Station_LatLongAr.csv**: This file contains the latitude, longitude, and area of all the HYDAT stations present in Ontario.

Please ensure these files are in your working directory before proceeding.

Change the path for directories where required. This is necessary when you're working with file inputs or outputs in your code. The exact path will depend on the file structure of your project and where your Jupyter notebook file is located.


In this code, `periods` is a list of tuples, where each tuple represents a period of interest. The `data` line is filtering the DataFrame to only include rows where the 'Date' is within the specified period. Remember to replace the dates with your actual period of interest.

<!-- # Define the period of interest
periods = [(1990, 2020)]
# Filter data for the period
data = data[(data['Date'] >= '1990-10-01') & (data['Date'] <= '2020-09-30')]   -->


# Mean Annual Discharge
1. It is determined by computing the arithmetic average of the daily streamflow values observed over the entire water year.
2. Mann Kendall and Pettit test is applied for trend direction and change sigificance.
3. Output will produce text files with results 
     - `meanQ_POR.csv`: This file contains the mean of each water year for the period of interest for each station.
        The stations are arranged in columns and the rows are years in increasing order.
    - `meanQ_station.csv`: This file contains the mean of each station over all the years.
    - 'MKnP_MeanAnnualD.csv': This file contains the results of Man kendall and Pettit test for Mean annual discharge.



In [ ]:
shapefile_path = r"C:\Users\TIWARIDI\OneDrive - Government of Ontario\Documents\provincial Boundary\SOBR_on84.shp"
base_map = gpd.read_file(shapefile_path)
data_folder = r"C:\Users\TIWARIDI\OneDrive - Government of Ontario\Documents\WSCdata\SOBR_FinalStations\SOBR_plot\CNP1970_2020"
fig_folder1 = os.path.join(data_folder ,"allSOBRplot")


# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder1, exist_ok=True)


mean_discharge_list = []
expected_months = set(range(1, 13))
# Directory containing the CSV files
csv_directory = 'C:/Users/TIWARIDI/OneDrive - Government of Ontario/Documents/WSCdata/SOBR_FinalStations/SOBR_plot/CNP1970_2020'

# Load the CSV files into dataframes
coordinates_df = pd.read_csv(r'C:\Users\TIWARIDI\OneDrive - Government of Ontario\Documents\WSCdata\SOBR_FinalStations\SOBR_plot\Station_LatLongAr.csv', sep='\t')


# Create a new directory for results
results_directory = 'C:/Users/TIWARIDI/OneDrive - Government of Ontario/Documents/WSCdata/SOBR_FinalStations/SOBR_plot/CNP1970_2020/Figurepettittann'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
    
fig_folderPTT = os.path.join(results_directory  ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)

# Get a list of CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_dfMAD = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance','Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound'])

# Define the period of interest
periods = [(1970, 2020)]

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file
    file_path = os.path.join(csv_directory, csv_file)
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
    station_number = csv_file.split('.')[0].upper() 

    # Filter data for the period 
    data = data[(data['Date'] >= '1970-10-01') & (data['Date'] <= '2020-09-30')]

    # Add a 'WaterYear' column to the data
    data['WaterYear'] = data['Date'].dt.year + (data['Date'].dt.month >= 10)
    
    
    for water_year, group in data.groupby('WaterYear'):
        # Extract the months for this WaterYear
        months_in_water_year = set(group['Date'].dt.month)

        # Check if any month is missing
        if months_in_water_year != expected_months:
            # Replace only the 'value' column in this WaterYear group with NaN
            data.loc[data['WaterYear'] == water_year, 'Value'] = np.nan


    mean_discharge = data.groupby('WaterYear')['Value'].apply(lambda x: x.dropna().mean())

    
    
    # Append the mean discharge for each water year to the list
    mean_discharge_list.append(mean_discharge)
    # Concatenate the list of mean discharge DataFrames into a single DataFrame
    concatenated_results = pd.concat(mean_discharge_list, axis=1)

   
    
    #perform men kendall test on mean annual discharge
    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(mean_discharge,lag=1)
    new_result = create_mk_result_entry(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept)
    results_dfMAD = pd.concat([results_dfMAD, new_result], ignore_index=True)

    locP, K, p_value = mk.pettitt_dip(mean_discharge.values)
    new_data = create_pettitt_result_entry(csv_file.replace('.csv', ''), K, p_value, locP, mean_discharge)
    results_df = pd.concat([results_df, new_data], ignore_index=True)
    
    
    ##################################################
    # Calculate mean before and after the change point
    change_point_year = mean_discharge.index[int(locP)]
    mean_before_change = mean_discharge[mean_discharge.index < change_point_year].mean()
    mean_after_change = mean_discharge[mean_discharge.index >= change_point_year].mean()
    
 
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(mean_discharge.index, mean_discharge, marker="o", color="black", s=8)
    plt.plot(mean_discharge.index, slope * np.arange(len(mean_discharge)) + intercept, color='red',label='Trend Line')
    plt.axvline(mean_discharge.index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=mean_discharge.index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=mean_discharge.index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Year",fontsize=6)
    plt.ylabel('Discharge (m^3/sec)',fontsize=6)

    plt.title(f"Mean Annual Flow - Station {station_number}",fontsize=6)             

        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {mean_discharge.index[int(locP)]}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{periods[0][0]}-{periods[0][1]}png")

    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)
    plt.close()


    ##################################################
         

# Save the DataFrame as a CSV file
results_csv_file = os.path.join(results_directory, 'results.csv')
results_df.to_csv(results_csv_file, index=False)

# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_df, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(results_directory, 'results_latlon.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')

merged_df1.to_csv(results_file_path1, index=False)


# Merge the dataframes based on the station name
merged_df = pd.merge( results_dfMAD, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path = os.path.join(results_directory, 'resultsMAD_latlon.csv')
merged_df = merged_df.drop_duplicates(subset='Station')
merged_df.to_csv(results_file_path, index=False)

column_means = concatenated_results.mean()

mean_results = pd.DataFrame(column_means, columns=['Mean'])
results_file_MEANQ = os.path.join(fig_folder1, 'meanQ_POR.csv')
concatenated_results .to_csv(results_file_MEANQ, index=False)
results_file_MEANstation = os.path.join(fig_folder1, 'meanQ_station.csv')
mean_results .to_csv(results_file_MEANstation, index=False)

#### adding %MAf to column
mean_results_reset = mean_results.reset_index(drop=True)
merged_df_reset = merged_df.reset_index(drop=True)

# Concatenating DataFrames side by side
merged_df11 = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "Slope as % of MAF"
merged_df11['Slope as % of MAF'] = abs((merged_df11['Slope'] / merged_df11['Mean']) * 100)


# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_df.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','Slope as % of MAF']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_MeanAnnualD.csv"), index=False)

The following lines of code are used to plot the figures for trend significance from the Mann-Kendall test and change significance from the Pettitt test.

In [ ]:
sb.plot_trend_mapPtt(merged_df1,shapefile_path,fig_folder1, 'Mean Annual Flow \n (1971-2020) \n Pettitt Significance', 'change_MAD.png')
sb.plot_trend_mapMK(merged_df11,shapefile_path,fig_folder1, 'Mean Annual Flow \n (1971-2020) \n MK Significance', 'Trend_MAD.png')

# Annual 3 day Maximum Flow magnitude
    - Calculated using a 3-day rolling window approach.
    - MKnP_3daymaxflow.csv: This file contains the results of Man kendall and Pettit test for 3 day maximum flow magnitude.

In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound'])

fig_folder = os.path.join(data_folder ,"Figure3dayflow7cate")

# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)
fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)

# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        
        station_number = csv_file.split('.')[0].upper() 
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()
        
 # Define the Water Year
        station_data['Water_Year'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1
                 
        station_data['3-day_rolling_max'] = station_data.groupby('Water_Year')['Value'].transform(lambda x: x.rolling(window=3, center=True,min_periods=1).mean())

####################################
        
            # Group by WaterYear
        for water_year, group in station_data.groupby('Water_Year'):
            # Extract the months for this WaterYear
#             months_in_water_year = set(group['Date'].dt.month)
            months_in_water_year = set(group.index.month)
#             print(months_in_water_year)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                # Replace only the 'value' column in this WaterYear group with NaN
                station_data.loc[station_data['Water_Year'] == water_year, 'Value'] = np.nan

        q_3daymaxn = station_data.groupby('Water_Year')['3-day_rolling_max'].apply(lambda x: x.dropna().max())
        q_3daymax = q_3daymaxn.drop([q_3daymaxn.index[0], q_3daymaxn.index[-1]])
     # Compute the Mann-Kendall trend

    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(q_3daymax,lag=1)

    new_result = create_mk_result_entry(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept)
    results_MK= pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(q_3daymax.values)

    locP, K, p_value = mk.pettitt_dip(q_3daymax.values[mask])
    new_data = create_pettitt_result_entry(csv_file.replace('.csv', ''), K, p_value, locP, q_3daymax)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
    
    
    ##################################################
    # Calculate mean before and after the change point
    change_point_year = q_3daymax.index[int(locP)]
    mean_before_change = q_3daymax[q_3daymax.index < change_point_year].mean()
    mean_after_change = q_3daymax[q_3daymax.index >= change_point_year].mean()

    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(q_3daymax.index, q_3daymax, marker="o", color="black", s=8)
    plt.plot(q_3daymax.index, slope * np.arange(len(q_3daymax)) + intercept, color='red',label='Trend Line')
    plt.axvline(q_3daymax.index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=q_3daymax.index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=q_3daymax.index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Year",fontsize=6)
    plt.ylabel('Discharge (m^3/sec)',fontsize=6)
    plt.title(f"3 day max flow - Station {station_number}",fontsize=6)
        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {q_3daymax.index[int(locP)]}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)
    plt.close()


    ##################################################
         


# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)


# Save the results to a CSV file
results_file_path = os.path.join(fig_folder, 'results.csv')
results_MK.to_csv(results_file_path, index=False)

data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "Slope as % of MAF"
result_df_all['Slope as % of MAF'] = abs((result_df_all['Slope'] / result_df_all['Mean']) * 100)


# Define the path for the output file
path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','Slope as % of MAF']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_3daymaxflow.csv"), index=False)


In [ ]:
sb.plot_trend_mapPtt(merged_df1, shapefile_path,fig_folder1,'3-day Maximum Flow \n (1971-2020) \n Pettitt Significance', 'change_3dayflow.png')
sb.plot_trend_mapMK(merged_df11,shapefile_path,fig_folder1, '3-day Maximum Flow \n (1971-2020) \n MK Significance', 'trend_3dayflow.png')


# Annual 3 day maximum flow timing of Year
    - Calculate 3-day rolling window approach and the corresponding timing is the 3-day maximum flow timing for the water year
    - MKnP_3daymaxflowTime.csv: This file contains the results of Man kendall and Pettit test for 3 day maximum flow magnitude.
    - result_df_all['slope over CNP'] = abs((result_df_all['Slope'] *30)) --------- In this line change the 30 to number of year to actual number of              years in the study, if is period of record then change to 50 , if climate normal period then keep 30


In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound','3-day_max','Day_of_year'])

# Specify the path to the folder containing the discharge CSV files
fig_folder = os.path.join(data_folder ,"Figure3dayflowTime7cate")
# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)

fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)

# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        #print(station_data)
        
        station_number = csv_file.split('.')[0].upper() 
       # print(station_number)
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()

        # Define the Water Year
        station_data['Water_Year'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1

      

        station_data['Water_Month'] = ((station_data.index.month + 2) % 12) + 1  
        
        
        # Convert index to string for concatenation
        index_str = station_data.index.strftime('%Y')

        # Calculate water day
        station_data['Water_Day'] = np.where(station_data.index.month >= 10,
                                             (station_data.index - pd.to_datetime(index_str + "-10-01")).days + 1,
                                             (station_data.index - pd.to_datetime((station_data.index.year - 1).astype(str) + "-10-01")).days + 1)

        # Adjust for leap years
        station_data['Water_Day'] = np.where((station_data.index.is_leap_year) & (station_data.index.month >= 10),
                                             station_data['Water_Day'] + 1,
                                             station_data['Water_Day'])


#         print(station_data)
        station_data['3-day_rolling_max'] = station_data.groupby('Water_Year')['Value'].transform(lambda x: x.rolling(window=3, center=True,min_periods=1).mean())
#         print(station_data)

      
    ####################################
        
            # Group by WaterYear
        for water_year, group in station_data.groupby('Water_Year'):

            months_in_water_year = set(group.index.month)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                station_data.loc[station_data['Water_Year'] == water_year, 'Value'] = np.nan
    
    #####################################

        max_flow_days_of_water_year = station_data.groupby('Water_Year')['3-day_rolling_max'].apply(lambda x: x.dropna().idxmax())


        max_flow_days_of_water_year = max_flow_days_of_water_year.dropna()
        max_flow_days_of_yearn = station_data.loc[max_flow_days_of_water_year]
        max_flow_days_of_year = max_flow_days_of_yearn.drop([max_flow_days_of_yearn.index[0], max_flow_days_of_yearn.index[-1]])

        max_flow_days_of_year['Day_of_year'] = max_flow_days_of_year.index.dayofyear
#         print( max_flow_days_of_water_year)
        max_flow_days_of_year['Month_of_year'] = max_flow_days_of_year.index.strftime('%m')
        max_day_of_year = max_flow_days_of_year.loc[max_flow_days_of_year['3-day_rolling_max'].idxmax()]
        max_day_value = max_day_of_year['3-day_rolling_max']
        corresponding_month = max_day_of_year['Month_of_year'] 
        corresponding_day = max_day_of_year['Day_of_year'] 

    # Compute the Mann-Kendall trend
    
    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(max_flow_days_of_year['Water_Day'],lag=1)

    new_result = create_mk_result_entrytime(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,max_day_value,corresponding_day,intercept)
    results_MK= pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(max_flow_days_of_year['Water_Day'].values)
    x =max_flow_days_of_year['Water_Day'].values[mask]
#     print(x)
    xx=max_flow_days_of_year['Water_Day']
    
#     print(len(max_flow_days_of_year))

    locP, K, p_value = mk.pettitt_dip(x)

    new_data = create_pettitt_result_entrytime(csv_file.replace('.csv', ''), K, p_value, locP, xx)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
    
        ##################################################
    # Calculate mean before and after the change point
    change_point_year = max_flow_days_of_year['Water_Day'].index[int(locP)]
    mean_before_change = max_flow_days_of_year['Water_Day'][max_flow_days_of_year['Water_Day'].index < change_point_year].mean()
    mean_after_change = max_flow_days_of_year['Water_Day'][max_flow_days_of_year['Water_Day'].index >= change_point_year].mean()
    
#     print(mean_before_change, mean_after_change)
  
    
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(max_flow_days_of_year['Water_Day'].index, max_flow_days_of_year['Water_Day'], marker="o", color="black", s=8)
    plt.plot(max_flow_days_of_year['Water_Day'].index, slope * np.arange(len(max_flow_days_of_year['Water_Day'])) + intercept, color='red',label='Trend Line')
    plt.axvline(max_flow_days_of_year['Water_Day'].index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=max_flow_days_of_year['Water_Day'].index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=max_flow_days_of_year['Water_Day'].index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Water Year",fontsize=6)
    plt.ylabel('Water Days',fontsize=6)
    plt.title(f"3 day max flow Timing - Station {station_number}",fontsize=6)
    
        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {max_flow_days_of_year['Water_Day'].index[int(locP)].year}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year+1}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)
#     plt.show()


    ##################################################

    
# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)

# Save the results to a CSV file
results_file_path = os.path.join(fig_folder, 'results.csv')
results_MK.to_csv(results_file_path, index=False)

data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "slope over CNP"
result_df_all['slope over CNP'] = abs((result_df_all['Slope'] *50))

path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','slope over CNP']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_3daymaxflowTime.csv"), index=False)
## plot

sb.plot_trend_mapPtt(merged_df1, shapefile_path,fig_folder1,'3-day Maximum Flow Timing\n (1971-2020) \n Pettitt Significance', 'change_3dayflowTime.png')
sb.plot_trend_mapMKT(merged_df11,shapefile_path,fig_folder1, '3-day Maximum Flow Timing\n (1971-2020) \n MK Significance', 'trend_3dayflowTime.png')


# Annual 7 day Minimum Flow magnitude
    - Calculated using a 7-day rolling window approach.
    - MKnP_7dayminflow.csv: This file contains the results of Man kendall and Pettit test for 7 day minimum flow magnitude.

In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound'])

# Specify the path to the folder containing the discharge CSV files
fig_folder = os.path.join(data_folder ,"Figure7dayflow7cate")

# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)

fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)


# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        #print(station_data)
        
        station_number = csv_file.split('.')[0].upper() 
       # print(station_number)
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()
        
         # Define the Water Year
        station_data['Water_Year'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1
        
                 
        station_data['7-day_rolling_min'] = station_data.groupby('Water_Year')['Value'].transform(lambda x: x.rolling(window=7, center=True,min_periods=1).mean())

            # Group by WaterYear
        for water_year, group in station_data.groupby('Water_Year'):

            months_in_water_year = set(group.index.month)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                # Replace only the 'value' column in this WaterYear group with NaN
                station_data.loc[station_data['Water_Year'] == water_year, 'Value'] = np.nan

    
    #####################################

        q_7dayminn = station_data[station_data['7-day_rolling_min'] > 0].groupby('Water_Year')['7-day_rolling_min'].apply(lambda x: x.dropna().min())

    
        q_7daymin = q_7dayminn.drop([q_7dayminn.index[0], q_7dayminn.index[-1]])

    # Perform the Mann-Kendall test
   
    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(q_7daymin,lag=1)

    new_result = create_mk_result_entry(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept)
    results_MK= pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(q_7daymin.values)

    locP, K, p_value = mk.pettitt_dip(q_7daymin.values[mask])
    new_data = create_pettitt_result_entry(csv_file.replace('.csv', ''), K, p_value, locP, q_7daymin)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
    
        ##################################################
    # Calculate mean before and after the change point
    change_point_year = q_7daymin.index[int(locP)]
    mean_before_change = q_7daymin[q_7daymin.index < change_point_year].mean()
    mean_after_change = q_7daymin[q_7daymin.index >= change_point_year].mean()
    
#     print(mean_before_change, mean_after_change)
  
    
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(q_7daymin.index, q_7daymin, marker="o", color="black", s=8)
    plt.plot(q_7daymin.index, slope * np.arange(len(q_7daymin)) + intercept, color='red',label='Trend Line')
    plt.axvline(q_7daymin.index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=q_7daymin.index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=q_7daymin.index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Water Year",fontsize=6)
    plt.ylabel('Discharge (m^3/sec)',fontsize=6)
    plt.title(f"7 day min flow - Station {station_number}",fontsize=6)
        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {q_7daymin.index[int(locP)]}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year+1}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)


    ##################################################
         

     
 # Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)


data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "Slope as % of MAF"
result_df_all['Slope as % of MAF'] = abs((result_df_all['Slope'] / result_df_all['Mean']) * 100)


# Define the path for the output file
path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','Slope as % of MAF']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_7dayminflow.csv"), index=False)
   
## plot
sb.plot_trend_mapPtt(merged_df1, shapefile_path,fig_folder1,'7-day Minimum Flow \n (1971-2020) \n Pettitt Significance', 'change_7dayflow.png')
sb.plot_trend_mapMK(merged_df11,shapefile_path,fig_folder1, '7-day Minimum Flow \n (1971-2020) \n MK Significance', 'trend_7dayflow.png')   



# Annual 7 day Minimum Flow timing
    - Calculate 7-day rolling window approach and the corresponding timing is the 7-day minimum flow timing for the water year
    - MKnP_7dayminflowTime.csv: This file contains the results of Man kendall and Pettit test for 7 day minimum flow magnitude.
      - result_df_all['slope over CNP'] = abs((result_df_all['Slope'] *30)) --------- In this line change the 30 to number of year to actual number of              years in the study, if is period of record then change to 50 , if climate normal period then keep 30

In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound','7-day_min','Day_of_year'])

# Specify the path to the folder containing the discharge CSV files
fig_folder = os.path.join(data_folder ,"Figure7dayflowTime7cate")

# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)

fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)


# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        #print(station_data)
        
        station_number = csv_file.split('.')[0].upper() 
#         print(station_number)
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()
        
        
        
        # Define the Water Year
        station_data['Water_Year'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1
        
        
        station_data['Water_Month'] = ((station_data.index.month + 2) % 12) + 1  
        
        
        # Convert index to string for concatenation
        index_str = station_data.index.strftime('%Y')

        # Calculate water day
        station_data['Water_Day'] = np.where(station_data.index.month >= 10,
                                             (station_data.index - pd.to_datetime(index_str + "-10-01")).days + 1,
                                             (station_data.index - pd.to_datetime((station_data.index.year - 1).astype(str) + "-10-01")).days + 1)

        # Adjust for leap years
        station_data['Water_Day'] = np.where((station_data.index.is_leap_year) & (station_data.index.month >= 10),
                                             station_data['Water_Day'] + 1,
                                             station_data['Water_Day'])


            
        station_data['7-day_rolling_min'] = station_data.groupby('Water_Year')['Value'].transform(lambda x: x.rolling(window=7, center=True,min_periods=1).mean())
#         print(station_data)

        ####################################
        
            # Group by WaterYear
        for water_year, group in station_data.groupby('Water_Year'):
            # Extract the months for this WaterYear
#             months_in_water_year = set(group['Date'].dt.month)
            months_in_water_year = set(group.index.month)
#             print(months_in_water_year)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                # Replace only the 'value' column in this WaterYear group with NaN
                station_data.loc[station_data['Water_Year'] == water_year, 'Value'] = np.nan
#                 print(station_data['Value'])
    
    #####################################
      


        filtered_data = station_data[station_data['7-day_rolling_min'] > 0]
        min_flow_days_of_water_year = filtered_data.groupby('Water_Year')['7-day_rolling_min'].apply(lambda x: x.dropna().idxmin())

         

        # Filter out missing values in the resulting index
        min_flow_days_of_water_year = min_flow_days_of_water_year.dropna()
        min_flow_days_of_yearn = station_data.loc[min_flow_days_of_water_year]
        min_flow_days_of_year = min_flow_days_of_yearn.drop([min_flow_days_of_yearn.index[0], min_flow_days_of_yearn.index[-1]])
        min_flow_days_of_year['Day_of_year'] = min_flow_days_of_year.index.dayofyear
    
        min_flow_days_of_year['Month_of_year'] = min_flow_days_of_year.index.strftime('%m')
        min_day_of_year = min_flow_days_of_year.loc[min_flow_days_of_year['7-day_rolling_min'].idxmin()]
        min_day_value = min_day_of_year['7-day_rolling_min']
        corresponding_month = min_day_of_year['Month_of_year'] 
        corresponding_day = min_day_of_year['Day_of_year'] 


    # Perform the Mann-Kendall test
   

    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(min_flow_days_of_year['Water_Day'],lag=1)

    new_result = create_mk_result_entrytime7(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,min_day_value,corresponding_day,intercept)
    results_MK= pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(min_flow_days_of_year['Water_Day'].values)
    x =min_flow_days_of_year['Water_Day'].values[mask]
    xx=min_flow_days_of_year['Water_Day']
#     print((x))
 

    locP, K, p_value = mk.pettitt_dip(min_flow_days_of_year['Water_Day'].values[mask])
    new_data = create_pettitt_result_entrytime(csv_file.replace('.csv', ''), K, p_value, locP, xx)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
    
        ##################################################
    # Calculate mean before and after the change point
    change_point_year = min_flow_days_of_year['Water_Day'].index[int(locP)]
    mean_before_change = min_flow_days_of_year['Water_Day'][min_flow_days_of_year['Water_Day'].index < change_point_year].mean()
    mean_after_change = min_flow_days_of_year['Water_Day'][min_flow_days_of_year['Water_Day'].index >= change_point_year].mean()
    
 
    
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(min_flow_days_of_year['Water_Day'].index, min_flow_days_of_year['Water_Day'], marker="o", color="black", s=8)
    plt.plot(min_flow_days_of_year['Water_Day'].index, slope * np.arange(len(min_flow_days_of_year['Water_Day'])) + intercept, color='red',label='Trend Line')
    plt.axvline(min_flow_days_of_year['Water_Day'].index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=min_flow_days_of_year['Water_Day'].index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=min_flow_days_of_year['Water_Day'].index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Water Year",fontsize=6)
    plt.ylabel('Water Day)',fontsize=6)
    plt.title(f"7 day min flow Timing - Station {station_number} ",fontsize=6)
       
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {min_flow_days_of_year['Water_Day'].index[int(locP)].year}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year+1}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)


    ##################################################
         


  
 
# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)


# Save the results to a CSV file
results_file_path = os.path.join(fig_folder, 'results.csv')
results_MK.to_csv(results_file_path, index=False)

data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "slope over CNP"
result_df_all['slope over CNP'] = abs((result_df_all['Slope'] *50))

# Display the updated DataFrame
#print(result_df_all)

#print(result_df)
# Define the path for the output file
path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','slope over CNP']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_7dayminflowTime.csv"), index=False)
## plot
sb.plot_trend_mapPtt(merged_df1, shapefile_path,fig_folder1, '7-day Minimum Flow Timing \n (1971-2020) \n Pettitt Significance', 'change_7dayflowTime.png')
sb.plot_trend_mapMKT(merged_df11, shapefile_path,fig_folder1, '7-day Minimum Flow Timing \n (1971-2020) \n MK Significance', 'trend_7dayflowTime.png')


# Annual Coefficient of Variation
    - Calculated by dividing the standard deviation of the annual streamflow values by the mean annual streamflow and expressed as a percentage.
    - MKnP_ACV.csv: This file contains the results of Man kendall and Pettit test for ACV.

In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','LowerBoundPtt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound'])

# Specify the path to the folder containing the discharge CSV files
fig_folder = os.path.join(data_folder ,"FigureAnnualCV7cate")

# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)

fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)


# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        #print(station_data)
        
        station_number = csv_file.split('.')[0].upper() 
       # print(station_number)
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()
        
          # Define the Water Year
        station_data['Water_Year'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1
        
        ####################################
        
            # Group by WaterYear
        for water_year, group in station_data.groupby('Water_Year'):
            # Extract the months for this WaterYear
            months_in_water_year = set(group.index.month)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                # Replace only the 'value' column in this WaterYear group with NaN
                station_data.loc[station_data['Water_Year'] == water_year, 'Value'] = np.nan
        annual_cvn = (station_data['Value'].resample('A-SEP').apply(lambda x: x.dropna().std()) /station_data['Value'].resample('A-SEP').apply(lambda x: x.dropna().mean()))

        annual_cv = annual_cvn.drop([annual_cvn.index[0], annual_cvn.index[-1]])

      
    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(annual_cv,lag=1)

    new_result = create_mk_result_entry(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept)
    results_MK= pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(annual_cv.values)

    locP, K, p_value = mk.pettitt_dip(annual_cv.values[mask])
    new_data = create_pettitt_result_entry(csv_file.replace('.csv', ''), K, p_value, locP,annual_cv)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
    
        ##################################################
    # Calculate mean before and after the change point
    change_point_year = annual_cv.index[int(locP)]
    mean_before_change = annual_cv[annual_cv.index < change_point_year].mean()
    mean_after_change = annual_cv[annual_cv.index >= change_point_year].mean()
    
#     print(mean_before_change, mean_after_change)
  
    
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(annual_cv.index, annual_cv, marker="o", color="black", s=8)
    plt.plot(annual_cv.index, slope * np.arange(len(annual_cv)) + intercept, color='red',label='Trend Line')
    plt.axvline(annual_cv.index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=annual_cv.index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=annual_cv.index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Water Year",fontsize=6)
    plt.ylabel('Discharge (m^3/sec)',fontsize=6)
#     plt.title(f"ACV - Station {station_number} and Change Point ({start_year+1}-{end_year})",fontsize=6)
    plt.title(f"ACV - Station {station_number})",fontsize=6)
        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {annual_cv.index[int(locP)]}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year+1}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)


    ##################################################
 
# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)

data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "Slope as % of MAF"
result_df_all['Slope as % of MAF'] = abs((result_df_all['Slope'] / result_df_all['Mean']) * 100)

# Display the updated DataFrame
#print(result_df_all)

#print(result_df)
# Define the path for the output file
path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','Slope as % of MAF']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_ACV.csv"), index=False)
   
## plot
sb.plot_trend_mapPtt(merged_df1,shapefile_path,fig_folder1, 'Annual Coefficient of Variation \n (1971-2020) \n Pettitt Significance', 'change_ACV.png')
sb.plot_trend_mapMK(merged_df11,shapefile_path,fig_folder1, 'Annual Coefficient of Variation \n (1971-2020) \n MK Significance', 'trend_ACV.png')   


# Richards Baker Flashiness Index
    - Calculated by the summing of the differences between flows on two consecutive data points and then dividing by the total average flow for a year.
    - MKnP_RBI.csv: This file contains the results of Man kendall and Pettit test for RBI.

In [ ]:
# Create an empty DataFrame to store the results
results_PT = pd.DataFrame(columns=['Station', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point'])

# Create an empty DataFrame to store the results
results_MK = pd.DataFrame(columns=['Station', 'Trend','Trend Direction', 'Trend Significance', 'Hypothesis', 'P-MK', 'Z-score', 'Taub', 'S', 'Variance of S', 'Slope', 'Intercept','Lower Bound','Upper Bound'])

# Specify the path to the folder containing the discharge CSV files
fig_folder = os.path.join(data_folder ,"FigureRBI7cate")

# Create the "figure" folder if it doesn't exist
os.makedirs(fig_folder, exist_ok=True)

fig_folderPTT = os.path.join(fig_folder ,"PTT")
os.makedirs(fig_folderPTT, exist_ok=True)

# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Loop through each CSV file in the folder
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    
    # Load the data from the CSV file
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    for period in periods:
        start_year, end_year = period

    # Select data for the specified time period
        station_data = data.loc[(data.index.year >= start_year) & (data.index.year <= end_year)]
        #print(station_data)
        
        station_number = csv_file.split('.')[0].upper() 
       # print(station_number)
        station_data =  station_data[ station_data['STATION_NUMBER'] == station_number].copy()
    
 
          # Define the Water Year
        station_data['WaterYear'] = station_data.index.where(station_data.index.month >= 10, station_data.index - pd.DateOffset(years=1)).year+1
        
        ####################################
        
            # Group by WaterYear
        for water_year, group in station_data.groupby('WaterYear'):
            # Extract the months for this WaterYear
#             months_in_water_year = set(group['Date'].dt.month)
            months_in_water_year = set(group.index.month)
#             print(months_in_water_year)

            # Check if any month is missing
            if months_in_water_year != expected_months:
                # Replace only the 'value' column in this WaterYear group with NaN
                station_data.loc[station_data['WaterYear'] == water_year, 'Value'] = np.nan
#                 print(station_data['Value'])

    

# Calculate the Richards-Baker Index over the annual period
    station_data['RBI'] = station_data.groupby('WaterYear')['Value'].apply(lambda group: group.diff().abs().sum() / group.sum())

#     rb_indexn =station_data.groupby('WaterYear')['Value'].apply(lambda group: group.diff().abs().sum() / group.sum())
    rb_indexn = station_data.groupby('WaterYear')['Value'].apply(lambda group: group.dropna().diff().abs().sum() / group.dropna().sum())

    rb_index = rb_indexn.drop([rb_indexn.index[0], rb_indexn.index[-1]])

    
    
#     print(rb_index)
    

    # Perform Mann-Kendall test on station_data_RBI_resampled

    trend, h, p, z, Tau, s, var_s, slope, intercept,lb,ub,Taub  = mk.yue_wang_modification_test(rb_index,lag=1)
    new_result = create_mk_result_entry(csv_file.replace('.csv', ''), trend, slope, h, p, z, Taub, s, var_s, lb, ub,intercept)
    results_MK = pd.concat([results_MK, new_result], ignore_index=True)
    
    mask = ~np.isnan(rb_index.values)

    locP, K, p_value = mk.pettitt_dip(rb_index.values[mask])
    new_data = create_pettitt_result_entry(csv_file.replace('.csv', ''), K, p_value, locP, rb_index)
    results_PT = pd.concat([results_PT, new_data], ignore_index=True)
        ##################################################
    # Calculate mean before and after the change point
    change_point_year = rb_index.index[int(locP)]
    mean_before_change = rb_index[rb_index.index < change_point_year].mean()
    mean_after_change = rb_index[rb_index.index >= change_point_year].mean()
    
#     print(mean_before_change, mean_after_change)
  
    
    plt.figure(figsize=(5, 4), facecolor='white')
    plt.scatter(rb_index.index, rb_index, marker="o", color="black", s=8)
    plt.plot(rb_index.index, slope * np.arange(len(rb_index)) + intercept, color='red',label='Trend Line')
    plt.axvline(rb_index.index[int(locP)], color='b', linestyle='--', label='Change Point')
    
    # Reduce the size of tick marks on x and y axes
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
  
    # Plot horizontal line for the mean before the change point
    plt.hlines(y=mean_before_change, xmin=rb_index.index.min(), xmax=change_point_year,color='green', linestyle='-', label=f'Mean Before ({mean_before_change:.3f})')

    # Plot horizontal line for the mean after the change point
    plt.hlines(y=mean_after_change, xmin=change_point_year, xmax=rb_index.index.max(),color='green', linestyle='-', label=f'Mean After ({mean_after_change:.3f})')


    plt.xlabel("Water Year",fontsize=6)
    plt.ylabel('Discharge (m^3/sec)',fontsize=6)
    plt.title(f"RBI - Station {station_number}",fontsize=6)    
        
    textstr = f"p-MK: {p:.3f}\nTau: {Tau:.3f}\nSens-Slope: {slope:.3f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.95, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    textstr = f"p-Pettitt: {p_value:.3f}\nChange point (CP): {rb_index.index[int(locP)]}\nMean before CP:{mean_before_change:.2f}\nMean after CP:{mean_after_change:.2f}"
    props = dict(boxstyle='round', facecolor='white', alpha=0.5, edgecolor='none')
    plt.text(1.01, 0.80, textstr, transform=plt.gca().transAxes, fontsize=6, verticalalignment='top', bbox=props)
        
    plt.legend(['Discharge Data', 'Trend Line', 'Change Point'], bbox_to_anchor=(1.01, 0), loc='lower left',fontsize=6)

    output_plot_filename = os.path.join(fig_folderPTT, f"Station_{station_number}_plot_{start_year+1}_{end_year}.png")
    plt.savefig(output_plot_filename, bbox_inches='tight', dpi=300)
#     plt.show()


    ##################################################
         



# Merge the dataframes based on the station name
merged_df1 = pd.merge( results_PT, coordinates_df, left_on="Station", right_on="Station")
# Define the path for the output file
results_file_path1 = os.path.join(fig_folder, 'results_latlonPT.csv')
merged_df1 = merged_df1.drop_duplicates(subset='Station')
merged_df1.to_csv(results_file_path1, index=False)


data_df =results_MK

# Merge the dataframes based on the station name
merged_df = pd.merge(data_df, coordinates_df, left_on="Station", right_on="Station")
merged_df = merged_df.drop_duplicates(subset='Station')

# Define the path for the output file
results_file_path = os.path.join(fig_folder, 'results_latlonMK.csv')
merged_df.to_csv(results_file_path, index=False)
merged_df_reset = merged_df.reset_index(drop=True)
mean_results_reset = mean_results.reset_index(drop=True)

# Concatenating DataFrames side by side
result_df_all = pd.concat([merged_df_reset, mean_results_reset], axis=1)
# Creating a new column "Slope as % of MAF"
result_df_all['Slope as % of MAF'] = abs((result_df_all['Slope'] / result_df_all['Mean']) * 100)

# Display the updated DataFrame
#print(result_df_all)

#print(result_df)
# Define the path for the output file
path = os.path.join(fig_folder, 'results_latlonMAF.csv')
result_df_all.to_csv(path, index=False)

merged_df11=result_df_all
# Assuming 'Station' is the common column in both dataframes
combined_MKnP = results_PT.merge(merged_df11, on='Station', how='outer')
selected_columns = ['Station','Latitude', 'Longitude', 'Pettitt_K', 'P-Ptt','Trend SignificancePtt', 'Change value','Change Point','Trend Direction', 'Trend Significance', 'P-MK', 'Taub', 'Slope', 'Lower Bound','Upper Bound','Area','Mean','Slope as % of MAF']
MKnP_MAD =combined_MKnP[selected_columns]
MKnP_MAD.to_csv(os.path.join(fig_folder1, "MKnP_RBI.csv"), index=False)
   
## plot
sb.plot_trend_mapPtt(merged_df1,shapefile_path,fig_folder1, 'Richards Baker Index \n (1971-2020) \n Pettitt Significance', 'change_RBI.png')
sb.plot_trend_mapMK(merged_df11,shapefile_path,fig_folder1, 'Richards Baker Index \n (1971-2020) \n MK Significance', 'trend_RBI.png')   
